In [72]:
!pip install pandas

In [194]:
import pandas
pandas.__version__

'1.4.2'

In [195]:
import pandas as pd
customers = "http://localhost:5678/customers"

shipping = "http://localhost:5679/shipping/orders"

products = "http://localhost:8000/sales/products"

In [196]:
carts_checkout_list = "http://localhost:8000/sales/checkout/list"

Create a one page dashboard or csv/excel export which shows
- customers original USD in wallet
- final USD in wallet
- whether they have ordered or failed to order
- the products they ordered

- For those who successfully ordered, output should show a column of statuses and the
time it took for each stage of a customer order.
- a summary of the time distribution (min/max/avg) of each stage of the order across all
customers.

In [197]:
lines=[]
line=[100,43.3,"failed","1 Tshirt, 2 Jeans","N/A"]
lines.append(line)
line=[100,43.3,"ordered","1 Tshirt, 2 Jeans","ordered=2h,delivered=1h"]
lines.append(line)
pd.DataFrame(lines,columns=["original_wallet","final_usd","order_status","ordered_products","shipping_time_info"])

,original_wallet,final_usd,order_status,ordered_products,shipping_time_info
0,100,43.3,failed,"1 Tshirt, 2 Jeans",N/A
1,100,43.3,ordered,"1 Tshirt, 2 Jeans","ordered=2h,delivered=1h"


In [198]:
import requests
json_customers=requests.get(customers).json()

df_customers=pd.read_json(customers)
customers_who_paid=df_customers[~df_customers.purchase_history.isnull()]

In [199]:
df_customers.iloc[0]

_id                                          6281ab07abe547c32c552ba1
full_name                                            Lianna Hartridge
wallet_usd                                                     690.52
purchase_history    [{'cart_id': '6281ab2ce64a86da3a0ca745', 'tota...
Name: 0, dtype: object

In [200]:
wallet_from_to_by_customer=pd.json_normalize(json_customers,record_path="purchase_history",meta=["_id","full_name","wallet_usd"]).groupby(["_id","full_name"]).agg({"from_wallet":"first","to_wallet":"last"})
wallet_from_to_by_customer

,,from_wallet,to_wallet
_id,full_name,,
6281ab07abe547c32c552ba1,Lianna Hartridge,990.36,690.52
6281ab07abe547c32c552ba2,Kaja Medendorp,845.48,547.88
6281ab07abe547c32c552ba3,Kalil Manton,1193.96,63.75
6281ab08abe547c32c552ba4,Tommy Grimwood,972.40,316.47
6281ab08abe547c32c552ba5,Reamonn Raybould,936.96,555.15
...,...,...,...
6281ab0cabe547c32c552c64,Alexandre Lyddon,1175.00,803.15
6281ab0cabe547c32c552c65,Michal Caldecutt,859.32,427.24
6281ab0cabe547c32c552c66,Donnie Escale,838.56,595.86


In [201]:
df_customers

,_id,full_name,wallet_usd,purchase_history
0,6281ab07abe547c32c552ba1,Lianna Hartridge,690.52,"[{'cart_id': '6281ab2ce64a86da3a0ca745', 'tota..."
1,6281ab07abe547c32c552ba2,Kaja Medendorp,547.88,"[{'cart_id': '6281ab2ee64a86da3a0ca746', 'tota..."
2,6281ab07abe547c32c552ba3,Kalil Manton,63.75,"[{'cart_id': '6281ab30e64a86da3a0ca747', 'tota..."
3,6281ab08abe547c32c552ba4,Tommy Grimwood,316.47,"[{'cart_id': '6281ab33e64a86da3a0ca748', 'tota..."
4,6281ab08abe547c32c552ba5,Reamonn Raybould,555.15,"[{'cart_id': '6281ab35e64a86da3a0ca749', 'tota..."
...,...,...,...,...
195,6281ab0cabe547c32c552c64,Alexandre Lyddon,803.15,"[{'cart_id': '6281ace1e64a86da3a0ca808', 'tota..."
196,6281ab0cabe547c32c552c65,Michal Caldecutt,427.24,"[{'cart_id': '6281ace3e64a86da3a0ca809', 'tota..."
197,6281ab0cabe547c32c552c66,Donnie Escale,595.86,"[{'cart_id': '6281ace5e64a86da3a0ca80a', 'tota..."
198,6281ab0cabe547c32c552c67,Ignace Pearmine,606.30,"[{'cart_id': '6281ace7e64a86da3a0ca80b', 'tota..."


In [202]:
pd.read_json(shipping)

,_id,cart_id,customer_id,status,products,status_history
0,6281ab2d36525d6dfd41c6d3,6281ab2ce64a86da3a0ca745,6281ab07abe547c32c552ba1,delivered,"[{'_id': None, 'name': 'Cheese - Parmigiano Re...","[{'status': 'ordered', 'created_at': '2022-05-..."
1,6281ab2f36525d6dfd41c6d4,6281ab2ee64a86da3a0ca746,6281ab07abe547c32c552ba2,delivered,"[{'_id': None, 'name': 'Cod - Fillets', 'price...","[{'status': 'ordered', 'created_at': '2022-05-..."
2,6281ab3136525d6dfd41c6d5,6281ab30e64a86da3a0ca747,6281ab07abe547c32c552ba3,delivered,"[{'_id': None, 'name': 'Muffin Mix - Raisin Br...","[{'status': 'ordered', 'created_at': '2022-05-..."
3,6281ab3436525d6dfd41c6d6,6281ab33e64a86da3a0ca748,6281ab08abe547c32c552ba4,delivered,"[{'_id': None, 'name': 'Hersey Shakes', 'price...","[{'status': 'ordered', 'created_at': '2022-05-..."
4,6281ab3636525d6dfd41c6d7,6281ab35e64a86da3a0ca749,6281ab08abe547c32c552ba5,delivered,"[{'_id': None, 'name': 'Pesto - Primerba, Past...","[{'status': 'ordered', 'created_at': '2022-05-..."
...,...,...,...,...,...,...
184,6281ace136525d6dfd41c78b,6281ace1e64a86da3a0ca808,6281ab0cabe547c32c552c64,delivered,"[{'_id': None, 'name': 'Mushroom - Chantrelle,...","[{'status': 'ordered', 'created_at': '2022-05-..."
185,6281ace436525d6dfd41c78c,6281ace3e64a86da3a0ca809,6281ab0cabe547c32c552c65,delivered,"[{'_id': None, 'name': 'Wine - Shiraz Wolf Bla...","[{'status': 'ordered', 'created_at': '2022-05-..."
186,6281ace636525d6dfd41c78d,6281ace5e64a86da3a0ca80a,6281ab0cabe547c32c552c66,delivered,"[{'_id': None, 'name': 'Spinach - Frozen', 'pr...","[{'status': 'ordered', 'created_at': '2022-05-..."
187,6281ace836525d6dfd41c78e,6281ace7e64a86da3a0ca80b,6281ab0cabe547c32c552c67,delivered,"[{'_id': None, 'name': 'Oil - Avocado', 'price...","[{'status': 'ordered', 'created_at': '2022-05-..."


In [203]:
df_carts=pd.read_json(carts_checkout_list)
df_carts

,_id,customer_id,status,products
0,6281ab2ce64a86da3a0ca745,6281ab07abe547c32c552ba1,success,"[{'_id': None, 'name': 'Cheese - Parmigiano Re..."
1,6281ab2ee64a86da3a0ca746,6281ab07abe547c32c552ba2,success,"[{'_id': None, 'name': 'Cod - Fillets', 'price..."
2,6281ab30e64a86da3a0ca747,6281ab07abe547c32c552ba3,success,"[{'_id': None, 'name': 'Muffin Mix - Raisin Br..."
3,6281ab33e64a86da3a0ca748,6281ab08abe547c32c552ba4,success,"[{'_id': None, 'name': 'Hersey Shakes', 'price..."
4,6281ab35e64a86da3a0ca749,6281ab08abe547c32c552ba5,success,"[{'_id': None, 'name': 'Pesto - Primerba, Past..."
...,...,...,...,...
195,6281ace1e64a86da3a0ca808,6281ab0cabe547c32c552c64,success,"[{'_id': None, 'name': 'Mushroom - Chantrelle,..."
196,6281ace3e64a86da3a0ca809,6281ab0cabe547c32c552c65,success,"[{'_id': None, 'name': 'Wine - Shiraz Wolf Bla..."
197,6281ace5e64a86da3a0ca80a,6281ab0cabe547c32c552c66,success,"[{'_id': None, 'name': 'Spinach - Frozen', 'pr..."
198,6281ace7e64a86da3a0ca80b,6281ab0cabe547c32c552c67,success,"[{'_id': None, 'name': 'Oil - Avocado', 'price..."


In [208]:
df_carts[df_carts.status!="success"]

,_id,customer_id,status,products
5,6281ab37e64a86da3a0ca74a,6281ab08abe547c32c552ba6,failed_by_insufficient_funds,"[{'_id': None, 'name': 'Oil - Sunflower', 'pri..."
20,6281ab59e64a86da3a0ca759,6281ab08abe547c32c552bb5,failed_by_insufficient_funds,"[{'_id': None, 'name': 'Anchovy Fillets', 'pri..."
50,6281ab9ce64a86da3a0ca777,6281ab09abe547c32c552bd3,failed_by_insufficient_funds,"[{'_id': None, 'name': 'The Pop Shoppe Pinappl..."
58,6281abaee64a86da3a0ca77f,6281ab09abe547c32c552bdb,failed_by_insufficient_funds,"[{'_id': None, 'name': 'Nantucket - Carrot Ora..."
60,6281abb2e64a86da3a0ca781,6281ab09abe547c32c552bdd,failed_by_insufficient_funds,"[{'_id': None, 'name': 'Spice - Greek 1 Step',..."
66,6281abc0e64a86da3a0ca787,6281ab09abe547c32c552be3,failed_by_insufficient_funds,"[{'_id': None, 'name': 'Kohlrabi', 'price': 25..."
98,6281ac07e64a86da3a0ca7a7,6281ab0aabe547c32c552c03,failed_by_insufficient_funds,"[{'_id': None, 'name': 'Barley - Pearl', 'pric..."
127,6281ac49e64a86da3a0ca7c4,6281ab0aabe547c32c552c20,failed_by_insufficient_funds,"[{'_id': None, 'name': 'Tofu - Firm', 'price':..."
147,6281ac76e64a86da3a0ca7d8,6281ab0babe547c32c552c34,failed_by_insufficient_funds,"[{'_id': None, 'name': 'Goulash Seasoning', 'p..."
155,6281ac87e64a86da3a0ca7e0,6281ab0babe547c32c552c3c,failed_by_insufficient_funds,"[{'_id': None, 'name': 'Beets', 'price': 299.9..."


In [214]:
df_customers.purchase_history.iloc[0]

[{'cart_id': '6281ab2ce64a86da3a0ca745',
  'total_purchase': 299.84,
  'from_wallet': 990.36,
  'to_wallet': 690.52}]

In [175]:
df_summary=wallet_from_to_by_customer.reset_index().set_index("_id").join(df_carts.set_index("customer_id")[["status","products"]])
df_summary.index.name="customer_id"
df_summary # here we need format product and add column shipping info

,full_name,from_wallet,to_wallet,status,products
customer_id,,,,,
6281a91fabe547c32c552ad9,Lianna Hartridge,990.36,586.12,success,"[{'_id': None, 'name': 'Tomatoes - Hot House',..."
6281a91fabe547c32c552ada,Kaja Medendorp,845.48,575.13,success,"[{'_id': None, 'name': 'Praline Paste', 'price..."
6281a91fabe547c32c552adb,Kalil Manton,1193.96,925.75,success,"[{'_id': None, 'name': 'Beef Flat Iron Steak',..."
6281a91fabe547c32c552adc,Tommy Grimwood,972.40,499.80,success,"[{'_id': None, 'name': 'Roe - Lump Fish, Red',..."
6281a91fabe547c32c552ade,Ethelda Prandoni,888.52,352.18,success,"[{'_id': None, 'name': 'Foam Cup 6 Oz', 'price..."
...,...,...,...,...,...
6281a923abe547c32c552b9a,Christie Rowlatt,1079.88,607.18,success,"[{'_id': None, 'name': 'Soup - Knorr, Veg / Be..."
6281a923abe547c32c552b9b,Glyn Carillo,1071.08,884.08,success,"[{'_id': None, 'name': 'Tofu - Firm', 'price':..."
6281a923abe547c32c552b9c,Alexandre Lyddon,1175.00,577.69,success,"[{'_id': None, 'name': 'Pork - Loin, Center Cu..."


In [176]:
",".join(map(str,[1,2,3]))

'1,2,3'

In [177]:
product_list=pd.json_normalize(df_summary["products"]).T.apply(lambda x: ",".join([row["name"] for row in x if not row is None])).tolist()

In [178]:
df_summary["products"] = product_list

In [179]:
df_summary

,full_name,from_wallet,to_wallet,status,products
customer_id,,,,,
6281a91fabe547c32c552ad9,Lianna Hartridge,990.36,586.12,success,"Tomatoes - Hot House,Bread - Raisin Walnut Ova..."
6281a91fabe547c32c552ada,Kaja Medendorp,845.48,575.13,success,Praline Paste
6281a91fabe547c32c552adb,Kalil Manton,1193.96,925.75,success,Beef Flat Iron Steak
6281a91fabe547c32c552adc,Tommy Grimwood,972.40,499.80,success,"Roe - Lump Fish, Red"
6281a91fabe547c32c552ade,Ethelda Prandoni,888.52,352.18,success,"Foam Cup 6 Oz,Pepper - Chilli Seeds Mild"
...,...,...,...,...,...
6281a923abe547c32c552b9a,Christie Rowlatt,1079.88,607.18,success,"Soup - Knorr, Veg / Beef,Muffin - Zero Transfat"
6281a923abe547c32c552b9b,Glyn Carillo,1071.08,884.08,success,"Tofu - Firm,Beans - Fava Fresh"
6281a923abe547c32c552b9c,Alexandre Lyddon,1175.00,577.69,success,"Pork - Loin, Center Cut,Anchovy In Oil,Anisett..."


# global stats by shipping order status

In [180]:
df_shipping=pd.read_json(shipping)


In [181]:
json_shipping = requests.get(shipping).json()

In [182]:
df_shipping

,_id,cart_id,customer_id,status,products,status_history
0,6281a99636525d6dfd41c615,6281a995e64a86da3a0ca295,6281a91fabe547c32c552ada,delivered,"[{'_id': None, 'name': 'Praline Paste', 'price...","[{'status': 'ordered', 'created_at': '2022-05-..."
1,6281a99636525d6dfd41c616,6281a995e64a86da3a0ca296,6281a91fabe547c32c552adb,delivered,"[{'_id': None, 'name': 'Beef Flat Iron Steak',...","[{'status': 'ordered', 'created_at': '2022-05-..."
2,6281a99736525d6dfd41c617,6281a995e64a86da3a0ca29a,6281a91fabe547c32c552ae1,delivered,"[{'_id': None, 'name': 'Cabbage - Red', 'price...","[{'status': 'ordered', 'created_at': '2022-05-..."
3,6281a99736525d6dfd41c618,6281a995e64a86da3a0ca299,6281a91fabe547c32c552adc,delivered,"[{'_id': None, 'name': 'Roe - Lump Fish, Red',...","[{'status': 'ordered', 'created_at': '2022-05-..."
4,6281a99736525d6dfd41c619,6281a995e64a86da3a0ca29d,6281a91fabe547c32c552adf,delivered,"[{'_id': None, 'name': 'Nantucket Cranberry Ju...","[{'status': 'ordered', 'created_at': '2022-05-..."
...,...,...,...,...,...,...
185,6281a9f936525d6dfd41c6ce,6281a9dde64a86da3a0ca356,6281a923abe547c32c552b9a,delivered,"[{'_id': None, 'name': 'Soup - Knorr, Veg / Be...","[{'status': 'ordered', 'created_at': '2022-05-..."
186,6281a9fa36525d6dfd41c6cf,6281a9dde64a86da3a0ca357,6281a923abe547c32c552b9b,delivered,"[{'_id': None, 'name': 'Tofu - Firm', 'price':...","[{'status': 'ordered', 'created_at': '2022-05-..."
187,6281a9fa36525d6dfd41c6d0,6281a9dee64a86da3a0ca358,6281a923abe547c32c552b9c,delivered,"[{'_id': None, 'name': 'Pork - Loin, Center Cu...","[{'status': 'ordered', 'created_at': '2022-05-..."
188,6281a9fb36525d6dfd41c6d1,6281a9e1e64a86da3a0ca35b,6281a923abe547c32c552ba0,delivered,"[{'_id': None, 'name': 'Sprouts - Corn', 'pric...","[{'status': 'ordered', 'created_at': '2022-05-..."


In [183]:

stages_orders=pd.json_normalize(json_shipping,record_path="status_history",meta=["_id","customer_id"])
stages_orders

,status,created_at,_id,customer_id
0,ordered,2022-05-16T01:32:06.545801,6281a99636525d6dfd41c615,6281a91fabe547c32c552ada
1,sent_to_warehouse,2022-05-16T03:58:06.545801,6281a99636525d6dfd41c615,6281a91fabe547c32c552ada
2,packaged,2022-05-16T13:21:06.545801,6281a99636525d6dfd41c615,6281a91fabe547c32c552ada
3,carrier_picked_up,2022-05-16T17:53:06.545801,6281a99636525d6dfd41c615,6281a91fabe547c32c552ada
4,out_for_delivery,2022-05-17T00:22:06.545801,6281a99636525d6dfd41c615,6281a91fabe547c32c552ada
...,...,...,...,...
1135,sent_to_warehouse,2022-05-16T16:25:47.580157,6281a9fb36525d6dfd41c6d2,6281a923abe547c32c552b9f
1136,packaged,2022-05-17T09:00:47.580157,6281a9fb36525d6dfd41c6d2,6281a923abe547c32c552b9f
1137,carrier_picked_up,2022-05-18T00:30:47.580157,6281a9fb36525d6dfd41c6d2,6281a923abe547c32c552b9f
1138,out_for_delivery,2022-05-18T15:02:47.580157,6281a9fb36525d6dfd41c6d2,6281a923abe547c32c552b9f


In [184]:
stages_orders["created_at"] = pd.to_datetime(stages_orders["created_at"])
stages_orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1140 entries, 0 to 1139
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   status       1140 non-null   object        
 1   created_at   1140 non-null   datetime64[ns]
 2   _id          1140 non-null   object        
 3   customer_id  1140 non-null   object        
dtypes: datetime64[ns](1), object(3)
memory usage: 35.8+ KB


In [185]:
control=stages_orders.groupby("_id").count()
control[control.status!=6]

,status,created_at,customer_id
_id,,,


In [186]:
stages_orders=stages_orders.sort_values(by=["_id","created_at"])
stages_orders

,status,created_at,_id,customer_id
0,ordered,2022-05-16 01:32:06.545801,6281a99636525d6dfd41c615,6281a91fabe547c32c552ada
1,sent_to_warehouse,2022-05-16 03:58:06.545801,6281a99636525d6dfd41c615,6281a91fabe547c32c552ada
2,packaged,2022-05-16 13:21:06.545801,6281a99636525d6dfd41c615,6281a91fabe547c32c552ada
3,carrier_picked_up,2022-05-16 17:53:06.545801,6281a99636525d6dfd41c615,6281a91fabe547c32c552ada
4,out_for_delivery,2022-05-17 00:22:06.545801,6281a99636525d6dfd41c615,6281a91fabe547c32c552ada
...,...,...,...,...
1135,sent_to_warehouse,2022-05-16 16:25:47.580157,6281a9fb36525d6dfd41c6d2,6281a923abe547c32c552b9f
1136,packaged,2022-05-17 09:00:47.580157,6281a9fb36525d6dfd41c6d2,6281a923abe547c32c552b9f
1137,carrier_picked_up,2022-05-18 00:30:47.580157,6281a9fb36525d6dfd41c6d2,6281a923abe547c32c552b9f
1138,out_for_delivery,2022-05-18 15:02:47.580157,6281a9fb36525d6dfd41c6d2,6281a923abe547c32c552b9f


In [187]:
stages_orders["time"] = stages_orders.created_at-stages_orders.created_at.shift(1)

In [188]:
stages_orders["time"]=stages_orders.time.shift(-1)

In [189]:
stats_global_by_status=stages_orders.groupby("status").agg({'time':['mean','min','max']})
stats_global_by_status=stats_global_by_status.drop("delivered")


# data to show in dashboard

In [190]:
stats_global_by_status.columns = ["mean_hours","min_hours","max_hours"]
stats_global_by_status["mean_hours"]=stats_global_by_status["mean_hours"].apply(lambda x: round(x.total_seconds()/60/60,2))
stats_global_by_status["min_hours"]=stats_global_by_status["min_hours"].apply(lambda x: round(x.total_seconds()/60/60,2))
stats_global_by_status["max_hours"]=stats_global_by_status["max_hours"].apply(lambda x: round(x.total_seconds()/60/60,2))

In [193]:
stats_global_by_status

,mean_hours,min_hours,max_hours
status,,,
carrier_picked_up,8.43,1.02,24.90
ordered,14.18,1.00,35.35
out_for_delivery,43.95,7.08,90.15
packaged,9.36,1.03,25.00
sent_to_warehouse,11.88,1.15,30.48


In [40]:
stats_global_by_status.columns.get_level_values(0)

Index(['mean', 'min', 'max'], dtype='object')

In [41]:
stages_orders_si = stages_orders.copy()

In [42]:
stages_orders_si=stages_orders_si[stages_orders_si.status!="delivered"]
stages_orders_si["label"]=stages_orders_si["status"] + ' = '+stages_orders_si["time"].astype(str)
stages_orders_si

,status,created_at,_id,customer_id,time,label
0,ordered,2022-05-15 21:30:36.270210,628170fc8b8db174cf91144b,628170d4ef228eb4f88b273d,0 days 00:00:00.197982,ordered = 0 days 00:00:00.197982
1,sent_to_warehouse,2022-05-15 21:30:36.468192,628170fc8b8db174cf91144b,628170d4ef228eb4f88b273d,0 days 00:00:00.109324,sent_to_warehouse = 0 days 00:00:00.109324
2,packaged,2022-05-15 21:30:36.577516,628170fc8b8db174cf91144b,628170d4ef228eb4f88b273d,0 days 00:00:00.031878,packaged = 0 days 00:00:00.031878
3,carrier_picked_up,2022-05-15 21:30:36.609394,628170fc8b8db174cf91144b,628170d4ef228eb4f88b273d,0 days 00:00:00.022817,carrier_picked_up = 0 days 00:00:00.022817
4,out_for_delivery,2022-05-15 21:30:36.632211,628170fc8b8db174cf91144b,628170d4ef228eb4f88b273d,0 days 00:00:00.123158,out_for_delivery = 0 days 00:00:00.123158
...,...,...,...,...,...,...
162,ordered,2022-05-15 21:31:41.609208,6281713d8b8db174cf911466,628170d5ef228eb4f88b275a,0 days 00:00:00.041015,ordered = 0 days 00:00:00.041015
163,sent_to_warehouse,2022-05-15 21:31:41.650223,6281713d8b8db174cf911466,628170d5ef228eb4f88b275a,0 days 00:00:00.052942,sent_to_warehouse = 0 days 00:00:00.052942
164,packaged,2022-05-15 21:31:41.703165,6281713d8b8db174cf911466,628170d5ef228eb4f88b275a,0 days 00:00:00.050131,packaged = 0 days 00:00:00.050131
165,carrier_picked_up,2022-05-15 21:31:41.753296,6281713d8b8db174cf911466,628170d5ef228eb4f88b275a,0 days 00:00:00.045012,carrier_picked_up = 0 days 00:00:00.045012


In [43]:
shipping_info=stages_orders_si.groupby("customer_id").agg({"label":';'.join})
shipping_info

,label
customer_id,
628170d4ef228eb4f88b273d,ordered = 0 days 00:00:00.197982;sent_to_wareh...
628170d5ef228eb4f88b273e,ordered = 0 days 00:00:00.060391;sent_to_wareh...
628170d5ef228eb4f88b273f,ordered = 0 days 00:00:00.041143;sent_to_wareh...
628170d5ef228eb4f88b2740,ordered = 0 days 00:00:00.039848;sent_to_wareh...
628170d5ef228eb4f88b2741,ordered = 0 days 00:00:00.035754;sent_to_wareh...
628170d5ef228eb4f88b2742,ordered = 0 days 00:00:00.054506;sent_to_wareh...
628170d5ef228eb4f88b2744,ordered = 0 days 00:00:00.061842;sent_to_wareh...
628170d5ef228eb4f88b2745,ordered = 0 days 00:00:00.065798;sent_to_wareh...
628170d5ef228eb4f88b2746,ordered = 0 days 00:00:00.038754;sent_to_wareh...


In [44]:
df_summary["shipping_info"] = shipping_info

# data to show in the dashboard

In [45]:
df_summary

,full_name,from_wallet,to_wallet,status,products,shipping_info
customer_id,,,,,,
628170d4ef228eb4f88b273d,Lianna Hartridge,990.36,561.04,success,Appetizer - Escargot Puff,ordered = 0 days 00:00:00.197982;sent_to_wareh...
628170d5ef228eb4f88b273e,Kaja Medendorp,845.48,176.09,success,"Eel - Smoked,Tart Shells - Sweet, 4,Scrubbie -...",ordered = 0 days 00:00:00.060391;sent_to_wareh...
628170d5ef228eb4f88b273f,Kalil Manton,1193.96,486.05,success,"Pork - Bacon, Double Smoked,Turnip - Mini",ordered = 0 days 00:00:00.041143;sent_to_wareh...
628170d5ef228eb4f88b2740,Tommy Grimwood,972.40,556.68,success,"Salmon - Atlantic, No Skin,Cheese - Grie Des C...",ordered = 0 days 00:00:00.039848;sent_to_wareh...
628170d5ef228eb4f88b2741,Reamonn Raybould,936.96,479.02,success,Soup - Campbells Beef Strogonoff,ordered = 0 days 00:00:00.035754;sent_to_wareh...
628170d5ef228eb4f88b2742,Ethelda Prandoni,888.52,141.25,success,"Oil - Cooking Spray,Cheese - Swiss Sliced,Broo...",ordered = 0 days 00:00:00.054506;sent_to_wareh...
628170d5ef228eb4f88b2744,Persis Carter,1134.32,410.02,success,"Cake - Lemon Chiffon,Tobasco Sauce,Capon - Bre...",ordered = 0 days 00:00:00.061842;sent_to_wareh...
628170d5ef228eb4f88b2745,Mellie Beams,1143.88,484.10,success,"Oil - Cooking Spray,Appetizer - Cheese Bites,S...",ordered = 0 days 00:00:00.065798;sent_to_wareh...
628170d5ef228eb4f88b2746,Zarla Fearnehough,1059.76,663.78,success,Tea - Herbal Sweet Dreams,ordered = 0 days 00:00:00.038754;sent_to_wareh...
